# Can we beat Welch's t-test?

The current version of this script is super-stupid. But to do any interesting research, you have to plant
a few small acorns, and this is the smallest of acorns.

Suppose you have to evaluate NUMBER_OF_EXPERIMENTS experiments very small experiment. By a weird
coincidence, they all have the same number of participants. There are CONTROL_GROUP_SIZE patients
getting a placebo, and EXPERIMENT_GROUP_SIZE getting a drug.

Unfortunately approximately PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING of those drugs actually have no effect either.

You want to create a classifier to determine whether or not the experimental drug worked.

Also, amazingly, the control group and the experimental group metrics are normally distributed. Obviously, the
experiments where the drug is useless have the same mean and standard deviation. In the experiments where
the drug is useful, they differ. The means of these datasets are uniformly distributed between 0-10. The standard deviations are uniformly distributed by 1-4.

Being a well-trained statistician with years of experience, you use Welch's t-test.

Your offsider is a clueless data scientist whose sole training consisted of reading the sklearn documentation. **Update (Diego): and some knowledge of neural networks**.

You have a future self who will implement a beautifully trained transformer or fully-connected neural network. (I haven't done that in this code yet. One day soon.)

Who wins?

In [35]:
CONTROL_GROUP_SIZE=5
EXPERIMENT_GROUP_SIZE=6
PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING=0.66
NUMBER_OF_EXPERIMENTS=100000

In [36]:
import scipy
import scipy.stats
import collections
import pandas
import sklearn.model_selection
import sklearn.svm
import sklearn.dummy
import sklearn.metrics
import sklearn.tree
import sklearn.ensemble
import sklearn.neighbors

In [37]:
Experiment = collections.namedtuple("Experiment",
                                    ["ControlLoc", "ControlScale", "ControlValues",
                                   "ExperimentLoc", "ExperimentScale", "ExperimentValues", 
                                   "ShouldShowResult"])

In [38]:
def generate_experiment():
    null_experiment = scipy.stats.uniform.rvs() <= PROPORTION_OF_EXPERIMENTS_THAT_DO_NOTHING
    control_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
    control_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    if null_experiment:
        experiment_loc = control_loc
        experiment_scale = control_scale
    else:
        experiment_loc = scipy.stats.uniform.rvs(loc=0, scale=10)
        experiment_scale = scipy.stats.uniform.rvs(loc=1, scale=4)
    control_values = scipy.stats.norm.rvs(loc=control_loc, scale=control_scale, size=CONTROL_GROUP_SIZE)
    experiment_values = scipy.stats.norm.rvs(loc=experiment_loc, scale=experiment_scale, size=EXPERIMENT_GROUP_SIZE)
    return Experiment(ControlLoc=control_loc, 
                      ControlScale=control_scale,
                     ControlValues=control_values,
                     ExperimentLoc=experiment_loc,
                     ExperimentScale=experiment_scale,
                     ExperimentValues=experiment_values,
                     ShouldShowResult=not(null_experiment))

def generate_experiments(number_of_experiments):
    return [generate_experiment() for n in range(number_of_experiments)]

In [39]:
def create_feature_dataframe(experiments):
    records = []
    for experiment in experiments:
        record = {}
        for i in range(CONTROL_GROUP_SIZE):
            record[f"c{i}"] = experiment.ControlValues[i]
        for i in range(EXPERIMENT_GROUP_SIZE):
            record[f"x{i}"] = experiment.ExperimentValues[i]
        records.append(record)
    return pandas.DataFrame.from_records(records)

def create_target_series(experiments):
    return pandas.Series([x.ShouldShowResult for x in experiments])

In [40]:
experiments = generate_experiments(NUMBER_OF_EXPERIMENTS)
create_feature_dataframe(experiments)

,c0,c1,c2,c3,c4,x0,x1,x2,x3,x4,x5
0,7.041426,9.532730,10.832903,7.917380,8.284008,9.239651,6.394088,9.516518,7.928147,9.760478,13.707901
1,10.509690,5.710724,5.155598,6.100354,-0.813376,6.698280,14.191196,8.167150,8.782613,7.868149,15.067801
2,7.112416,6.070608,5.325655,8.020194,12.345723,8.907949,-1.508427,14.320921,9.930717,11.813849,9.808185
3,4.253879,5.790526,4.202894,7.733208,5.190561,1.123563,2.561735,5.518133,7.621124,4.742116,9.220730
4,4.990388,3.996352,11.128080,7.319655,8.475199,7.467409,10.972181,3.406581,4.996548,8.795480,9.100131
...,...,...,...,...,...,...,...,...,...,...,...
99995,2.672116,6.799567,5.515955,8.013847,5.573475,1.267757,3.506946,7.952793,6.528013,5.525441,10.482136
99996,5.274091,2.835349,6.595077,3.479650,2.761597,4.018028,6.553090,5.150932,3.197582,2.524040,0.455807
99997,8.508778,8.163286,8.091643,8.712101,8.670028,11.308870,11.651300,10.201722,10.209395,7.768742,11.368282
99998,3.267405,3.628479,7.119019,6.483939,3.202671,6.547811,2.650536,5.738234,5.217257,4.371338,8.654002


In [41]:
create_target_series(experiments).mean()

0.33986

In [42]:
class WelchTTest:
    def __init__(self):
        pass
    def fit(self, X,y, weights=[]):
        pass
    def predict(self, Xs):
        answer = []
        for experiment in Xs:
            outcome = scipy.stats.ttest_ind(experiment.ControlValues, experiment.ExperimentValues, equal_var=False)
            if outcome.pvalue < 0.05:
                answer.append(True)
            else:
                answer.append(False)
        return answer

In [43]:
welch = WelchTTest()
welch_answers = welch.predict(experiments)
print(sklearn.metrics.confusion_matrix(create_target_series(experiments), welch_answers))

[[63035  2979]
 [20901 13085]]


In [44]:
sklearn.metrics.confusion_matrix([False, False, False, False, True, True, True, True],
                                [True, False, False, False, True, True, True, True])[0][1]

1

In [45]:
def type_1_error_ratio(y_true, y_pred):
    return sklearn.metrics.confusion_matrix(y_true, y_pred)[0][1] / len(y_true)

type_1_error_score = sklearn.metrics.make_scorer(type_1_error_ratio, greater_is_better=False)

## The target to beat...

For accuracy.

In [46]:
welch_accuracy = sklearn.metrics.accuracy_score(create_target_series(experiments), welch_answers)
welch_accuracy

0.7612

But you have to keep the type 1 error rate below this...

In [47]:
welch_type_1_error_ratio = type_1_error_ratio(create_target_series(experiments), welch_answers)
welch_type_1_error_ratio

0.02979

## Various stupid ways to beat it

Dummy gives us a baseline to be sure that the real models aren't just predicting the most common case.

In [48]:
dummy_scores = sklearn.model_selection.cross_validate(sklearn.dummy.DummyClassifier(strategy='most_frequent'), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                      )
print("Mean accuracy:", dummy_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(dummy_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.66014
Mean type 1 error: 0.0


## Neural Network

In [49]:
import numpy as np
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [50]:
tf.__version__

'2.2.0'

In [51]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [52]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)

In [53]:
%%time
inputs = create_feature_dataframe(experiments).to_numpy(dtype=float)
targets = create_target_series(experiments).to_numpy(dtype=float)
max_epochs = 300
hidden_size = 10
dropout_rate = 0.2
batch_size = 512
fold = 1
accuracies = []
type_1_errors = []
for train, test in kfold.split(inputs,targets):
    print("Fold", fold)

    model = models.Sequential()
    model.add(layers.Dense(hidden_size, activation='relu',input_shape=(11,)))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)
    history = model.fit(inputs[train], targets[train],
                    epochs=max_epochs,
                    batch_size=batch_size,
                    validation_split=0.2,
                    callbacks=[earlystopping],
                    verbose=0)
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    predictions = model.predict(inputs[test])[:,0] >= 0.5 # Predict 1 if the probability is >= 0.5
    type_1_error = type_1_error_ratio(targets[test], predictions)
    print("After %i epochs out of %i" % (len(history.history['accuracy']), max_epochs))
    print("%s of fold %i: %f" % (model.metrics_names[0], fold, scores[0]))
    print("%s of fold %i: %f" % (model.metrics_names[1], fold, scores[1]))
    print("Type 1 error ratio of fold %i: %f" % (fold, type_1_error))
    print()
    accuracies.append(scores[1])
    type_1_errors.append(type_1_error)
    fold += 1
print("Mean accuracy:", np.mean(accuracies))
print("Mean type 1 error ratio:", np.mean(type_1_errors))

Fold 1
After 20 epochs out of 300
loss of fold 1: 0.508105
accuracy of fold 1: 0.771750
Type 1 error ratio of fold 1: 0.035650

Fold 2
After 41 epochs out of 300
loss of fold 2: 0.510568
accuracy of fold 2: 0.768550
Type 1 error ratio of fold 2: 0.039350

Fold 3
After 26 epochs out of 300
loss of fold 3: 0.510944
accuracy of fold 3: 0.769800
Type 1 error ratio of fold 3: 0.038400

Fold 4
After 30 epochs out of 300
loss of fold 4: 0.510418
accuracy of fold 4: 0.769200
Type 1 error ratio of fold 4: 0.036750

Fold 5
After 21 epochs out of 300
loss of fold 5: 0.515391
accuracy of fold 5: 0.769850
Type 1 error ratio of fold 5: 0.036600

Mean accuracy: 0.7698300004005432
Mean type 1 error ratio: 0.03735
CPU times: user 1min 48s, sys: 10.6 s, total: 1min 59s
Wall time: 1min 16s


## Random forest 

It seems to beat Welch if there is enough training data.

In [54]:
%%time
rfc_scores = sklearn.model_selection.cross_validate(sklearn.ensemble.RandomForestClassifier(), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                               )
print("Mean accuracy:", rfc_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(rfc_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.78435
Mean type 1 error: 0.03076
CPU times: user 3min 36s, sys: 996 ms, total: 3min 37s
Wall time: 3min 37s


In [55]:
if abs(rfc_scores['test_type_1_error_ratio'].mean()) < 0.05 and rfc_scores['test_accuracy'].mean() > welch_accuracy:
    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio


In [56]:
if abs(rfc_scores['test_type_1_error_ratio'].mean()) < welch_type_1_error_ratio:
    print("Random forest beat the Welch t-test on type 1 errors")

## Neighbour methods

In [57]:
%%time
knn_scores = sklearn.model_selection.cross_validate(sklearn.neighbors.KNeighborsClassifier(), 
                                       create_feature_dataframe(experiments),
                                       create_target_series(experiments),
                                       scoring={'accuracy': 'accuracy', 
                                                'type_1_error_ratio': type_1_error_score}
                                               )
print("Mean accuracy:", knn_scores['test_accuracy'].mean())
print("Mean type 1 error:", abs(knn_scores['test_type_1_error_ratio'].mean()))

Mean accuracy: 0.7626999999999999
Mean type 1 error: 0.08318
CPU times: user 1min 25s, sys: 136 ms, total: 1min 25s
Wall time: 1min 25s


In [58]:
if abs(knn_scores['test_type_1_error_ratio'].mean()) < 0.05 and knn_scores['test_accuracy'].mean() > welch_accuracy:
    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

In [59]:
#%%time
#
#radnn_scores = sklearn.model_selection.cross_validate(sklearn.neighbors.RadiusNeighborsClassifier(), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments),
#                                       scoring={'accuracy': 'accuracy', 
#                                                'type_1_error_ratio': type_1_error_score}
#                                               )
#print("Mean accuracy:", radnn_scores['test_accuracy'].mean())
#print("Mean type 1 error:", abs(radnn_scores['test_type_1_error_ratio'].mean()))

In [60]:
#if abs(radnn_scores['test_type_1_error_ratio'].mean()) < 0.05 and radnn_scores['test_accuracy'].mean() > welch_accuracy:
#    print("Random forest classifier beat the Welch t-test while maintaining a low type 1 error ratio")

### Lost causes

You would think that SVM models would work really well; but they take too long to be practical. It's also
not clear whether they are actually improving with more data.

In [61]:
#%%time
#sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='rbf', C=1e9), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()

In [62]:
#%%time
#sklearn.model_selection.cross_validate(sklearn.svm.SVC(kernel='poly', C=1e9, degree=2), 
##                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()

In [63]:
#sklearn.model_selection.cross_validate(sklearn.svm.LinearSVC(dual=True, C=1e9), 
#                                       create_feature_dataframe(experiments),
#                                       create_target_series(experiments))['test_score'].mean()